# Plotting

A good way to illustrate the plotting possibilities is through a long list of demos.

In [ ]:
import osyris
import numpy as np
import matplotlib.pyplot as plt

data = osyris.Dataset(71, scale="au", path="osyrisdata").load()
ind = np.argmax(data["hydro"]["density"])
center = data["amr"]["xyz"][ind.values]

### 1. A 2D histogram of density vs magnetic field

By default, the `histogram` function will show a binned count of cells

In [ ]:
osyris.histogram(data["hydro"]["density"], data["hydro"]["B_field"],
                 norm="log", loglog=True)

Additional components can be overlayed using `layers`:

In [ ]:
osyris.histogram(data["hydro"]["density"], data["hydro"]["B_field"],
                 {"data": data["hydro"]["mass"], "norm": "log"}, # layer 1
                 {"data": data["amr"]["level"], "operation": "mean", "mode": "contour", "colors": "k"}, # layer 2
                 loglog=True)

### 2. Simple cut plane

Create a 2D gas density slice 100 au wide through the plane normal to `z`,
with velocity vectors overlayed as arrows, once agains using `layers`:

In [ ]:
osyris.plane({"data": data["hydro"]["density"], "norm": "log"}, # layer 1
             {"data": data["hydro"]["velocity"], "mode": "vec"}, # layer 2
             dx=100 * osyris.units("au"),
             origin=center,
             direction="z")

### 3. Cut plane at an arbitrary angle

The `plane` function will also accept a vector to define the normal direction to the plane

In [ ]:
osyris.plane({"data": data["hydro"]["density"], "norm": "log"},
             {"data": data["hydro"]["velocity"], "mode": "vec"},
             dx=100 * osyris.units("au"),
             origin=center,
             direction=[-1, 1, 1])

### 4. Automatic “top/side” slice orientation according to angular momentum

Create a 2D slice of the logarithm of density 50 au wide using automatic orientation based on the angular momentum in the data.
This is useful for looking at disks.
Use the `"top"` direction for the slice to view the disk from above

In [ ]:
osyris.plane({"data": data["hydro"]["density"], "norm": "log"},
             {"data": data["hydro"]["velocity"], "mode": "vec"},
             dx=50 * osyris.units("au"),
             origin=center,
             direction="top")

Use the `direction="side"` for the slice to view the disk from the side

In [ ]:
osyris.plane({"data": data["hydro"]["density"], "norm": "log"},
             {"data": data["hydro"]["velocity"], "mode": "vec"},
             dx=50 * osyris.units("au"),
             origin=center,
             direction="side")

### 5. Embedding plots in existing matplotlib axes

In this example, we create two subplot axes with `matplotlib`.

Next, we plot in the left panel the log of density as a coloured slice with velocity vectors.
The minimum and maximum of $\log(\rho)$ is forced to `-14` and `-9`.
We give the `plane` call the axes to use via the `ax` argument.
Next, we overlay some custom chosen density contours with different line styles and colours.

In the right panel, we plot a plane of temperature and overlay some lightgray contours showing the AMR levels.
We specify only integer contour levels.

In [ ]:
# Create figure
fig = plt.figure(figsize=(8, 3))
ax1 = fig.add_subplot(121)
ax2 = fig.add_subplot(122)
plt.subplots_adjust(wspace=0.5)

# Define region to plot
dx = 15.0 * osyris.units("au")

# Left plot: coloured density slice with overlayed contours
osyris.plane({"data": data["hydro"]["density"], "norm": "log",
                               "vmin": 1.0e-14,
                               "vmax": 1.0e-9},
             {"data": data["hydro"]["velocity"], "mode": "vec"},
             {"data": data["hydro"]["density"], "mode": "contour",
                  "levels": [1.0e-12,1.0e-11,1.0e-9],
                  "colors": ('yellow','k',"lime"),
                  "linewidths": [2,5,2],
                  "linestyles": ["solid","dashed","solid"],
                  "cmap": None,
                  "labels": False},
             dx=dx,
             origin=center,
             direction="z", ax=ax1)

osyris.plane({"data": data["hydro"]["temperature"], "norm": "log", "mode": "contourf",
              "levels": np.logspace(1.4, 3, 21), "cmap": "hot"},
             {"data": data["amr"]["level"], "mode": "contour", "colors": "w", "levels": [14, 15, 16]},
             dx=dx,
             origin=center,
             direction="z", ax=ax2)

### 6. Plot only a subset of cells belonging to a disk

In this example, we select cells according to their density and plot only those.
This is done by creating a new field and using the `numpy` `where` function.
To combine more than one selection criteria, use the `logical_and` `numpy` function.

This is useful for plotting disks around protostars, for example.
Here we select the cells with a density in the range $-12.5 < \log(\rho) < -11.0$.

In [ ]:
data["hydro"]["disk_density"] = osyris.Array(
    np.ma.masked_where(np.logical_or(
        data["hydro"]["density"].values < 3.0e-13,
        data["hydro"]["density"].values > 1.0e-11),
                       data["hydro"]["density"].values),
                       unit=data["hydro"]["density"].unit)
osyris.plane(data["hydro"]["disk_density"], dx=50 * osyris.units("au"),
             norm="log", origin=center, mode="image")

Computing the disk mass can be achieved via

In [ ]:
np.sum(data["hydro"]["disk_density"]*(data["amr"]["dx"]**3)).to("msun")

### 7. Difference between two snapshots

Here, we want to make a map of the difference in density between two snapshots.
Because we do not necessarily have the same number of cells at the same place,
we first have to make uniform 2D maps using the `plane` function,
which we can then directly compare.

The `plane` function actually returns an object that contains the raw data that was used to create the figure.
By using the `plot=False` argument, we can silence the figure generation, and use the data in a custom figure.

**Note:** For this to make sense, the two outputs have to be centered around the same center point.

In [ ]:
# Read data from a later snapshot
data2 = osyris.Dataset(201, scale="au", path="osyrisdata").load()

In [ ]:
dx = 100 * osyris.units("au")
# Extract density slices by copying data into structures
plane1 = osyris.plane({"data": data["hydro"]["density"], "norm": "log"},
                      dx=dx, origin=center, direction="z", plot=False)

plane2 = osyris.plane({"data": data2["hydro"]["density"], "norm": "log"},
                      dx=dx, origin=center, direction="z", plot=False)

# Get x,y coordinates
x = plane1.x
y = plane1.y

# Density difference
rho1 = np.log10(plane1.layers[0]["data"])
rho2 = np.log10(plane2.layers[0]["data"])
diff = (rho1 - rho2) / rho2

# Create figure
fig, ax = plt.subplots()
cf = ax.contourf(x, y , diff, cmap='RdBu',
                 levels=np.linspace(-0.12,0.12,31))
ax.set_aspect("equal")
cb = plt.colorbar(cf, ax=ax)
cb.ax.set_ylabel("Relative difference")

### 8. Slice above the origin

We want to plot a slice of density but through a point which is 5 AU above the centre of the domain,
defined as the cell with the highest density.
This is done by setting the `origin` coordinate to `[0, 0, 5]`

In [ ]:
origin = center.copy()
origin.values[-1] += 5
osyris.plane({"data": data["hydro"]["density"], "norm": "log"},
             {"data": data["hydro"]["velocity"], "mode": "vec"},
             dx=dx,
             origin=origin,
             direction="z")